In [1]:
import pandas as pd 
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
# Import the displaCy library
import texthero as hero

In [2]:
# Reading the files
myfile = open('snapchat.txt')
myfile_text = myfile.read().lower()



In [3]:
# Ambiguity Terms
Condition = ['depending', 'necessary', 'appropriate',
                 'inappropriate', 'as needed', 'as applicable', 'when applicable', 'if applicable',
                   'where applicable', 'when needed', 'if needed', 'where needed' 'reasonable', 'reasonably'
                 'otherwise reasonably', 'sometimes',
                 'from time to time']
Generalization = ['generally', 'mostly', 'widely',
                  'general', 'commonly',
                  'usually', 'normally', 'typically',
                  'largely', 'often', 'primarily',
                  'among other things']
Modality = ['may', 'might', 'can', 'could', 'would',
            'likely', 'possible', 'possibly']
Numeric_quantifier = ['anyone', 'certain', 'everyone',
                      'numerous', 'some', 'most', 'few',
                      'much', 'many', 'various', 'include',
                      'limited to', 
                      'including but not limited to']
keepWords = (Condition + Generalization + Modality + Numeric_quantifier)

ambiguousWords =(['depending', 'necessary', 'appropriate',
                 'inappropriate', 'as_needed', 'as_applicable', 'when_applicable', 'if_applicable',
                   'where_applicable', 'when_needed', 'if_needed', 'where_needed', 'reasonable', 'reasonably'
                 'otherwise_reasonably', 'sometimes',
                 'from_time_to_time']+['generally', 'mostly', 'widely',
                  'general', 'commonly',
                  'usually', 'normally', 'typically',
                  'largely', 'often', 'primarily',
                  'among other things']+['may', 'might', 'can', 'could', 'would',
            'likely', 'possible', 'possibly']+['anyone', 'certain', 'everyone',
                      'numerous', 'some', 'most', 'few',
                      'much', 'many', 'various', 'include',
                      'limited_to', 
                      'including_but_not_limited to'])


In [4]:
def len_str(x):
  return len(x)

def replace_word(orig_text, matcher):
    tok = nlp(orig_text)
    text = ''
    buffer_start = 0
    for word, match_start, match_end in matcher(tok):
        if match_start > buffer_start:  # If we've skipped over some tokens, let's add those in (with trailing whitespace if available)
            text += tok[buffer_start: match_start].text + tok[match_start - 1].whitespace_
        text += nlp.vocab.strings[word] + tok[match_start].whitespace_  # Replace token, with trailing whitespace if available
        buffer_start = match_end
    if buffer_start < len(tok):
        text += tok[buffer_start:].text
    return text

def ReplaceKeepwords(orig_text):
    global keepWords
    temp = keepWords.sort(key=len_str,reverse=True)
    text = orig_text
    # matcher = Matcher(nlp.vocab)
    for i in keepWords:
        matcher = Matcher(nlp.vocab)
        replacement = "_".join(i.split())
        rule = [ {"LOWER": j} for j in i.split() ]
        matcher.add(replacement, None, rule)
        text = replace_word(text, matcher)
    return text



In [5]:
final_text = nlp(ReplaceKeepwords(myfile_text))

In [6]:
# getting all sentences containing ambiguity terms
phrase = []



def get_sentences(sentence, phrase):
                 
    for token in sentence:
        count = 0
        for term in ambiguousWords:
            if token.text == term:
                count =1;
        if count == 1:
            phrase.append(sentence)
                
            
    
for sent in final_text.sents:
    get_sentences(sent, phrase)
print(len(phrase))

154


In [7]:
# creating a data frame
df = pd.DataFrame()
df['Phrase'] = phrase

for amb in ambiguousWords:
    c1 =[]
    for i in phrase:
        count = 0
        for token in i:
            if token.text == amb:
                count  = 1

        c1.append(count)
    df[amb] = c1
    

In [8]:
df.to_csv("snapchat.csv")

In [9]:
for token in df['Phrase'][0]:
    print(f'{token.text:{10}} {token.pos_:{8}} {token.tag_:{6}} {spacy.explain(token.tag_)}')

you        PRON     PRP    pronoun, personal
use        VERB     VBP    verb, non-3rd person singular present
these      DET      DT     determiner
services   NOUN     NNS    noun, plural
,          PUNCT    ,      punctuation mark, comma
you        PRON     PRP    pronoun, personal
’ll        VERB     MD     verb, modal auxiliary
share      VERB     VB     verb, base form
some       DET      DT     determiner
information NOUN     NN     noun, singular or mass
with       ADP      IN     conjunction, subordinating or preposition
us         PRON     PRP    pronoun, personal
.          PUNCT    .      punctuation mark, sentence closer


In [10]:

count = 0
b =1000

for token in df['Phrase'][0]:
            
    if (token.pos_ == 'VERB'):
        ambiguousVerb = token
        
            
    for term in ambiguousWords:
        if token.text == term:
            print(ambiguousVerb)
            print(term)
            b = count
            


    if (token.pos_ == 'NOUN' and ((count > b) and (count<=b+2))):
            print(token)
    count +=1





share
some
information


In [48]:
df["Combinations"] = df. sum(axis=1)

In [49]:
df.loc['Column_Total']= df.sum(numeric_only=True, axis=0)


In [14]:
from texthero import preprocessing
my_pipe = [preprocessing.fillna,
          preprocessing.remove_diacritics,
          preprocessing.remove_whitespace]

In [17]:
df['clean-text'] = hero.clean(df['Phrase'], my_pipe)

df['clean-text'] = [n.replace(')', '') for n in df['clean-text']]
df['clean-text'] = [n.replace('(', '') for n in df['clean-text']]
df['clean-text'] = [n.replace('{', '') for n in df['clean-text']]
df['clean-text'] = [n.replace('}', '') for n in df['clean-text']]

In [18]:
df['TFIDF'] = hero.tfidf(df['clean-text'], max_features =2900)

In [19]:
#2 Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [20]:
#using Tagged documents to tag the Document
tagged_docs = [TaggedDocument(doc.split(' ') ,[i]) for i, doc in enumerate(df['clean-text'])]

model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)

model.build_vocab(tagged_docs)

model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)

card2vec = [model.infer_vector((df['clean-text'][i].split(' '))) for i in range(0,len(df['clean-text']))]

df['Doc2vec'] = np.array(card2vec).tolist()

In [24]:
#Universal sentence encoder
import tensorflow_hub as hub

#download the model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [25]:
embeddings = embed(df['clean-text'])#create list from np arrays
use= np.array(embeddings).tolist()#add lists as dataframe column
df['use'] = use

#t-SNE algorithm for convertung all vectors in 2 dimensions
df['tsnetfidf'] = hero.tsne(df['TFIDF'])
df['tsnec2v'] = hero.tsne(df['Doc2vec'])
df['tsneuse'] = hero.tsne(df['use'])

df[['tsnetfidf','tsnec2v', 'tsneuse']].head()

,tsnetfidf,tsnec2v,tsneuse
Phrase,,,
"(you, use, these, services, ,, you, ’ll, share, some, information, with, us, .)","[0.5357636213302612, 13.304076194763184]","[-13.187685012817383, 4.814410209655762]","[2.5985493659973145, 3.0634214878082275]"
"(and, it, ’s, why, we, ’ve, tried, to, write, it, in, a, way, that, ’s, blissfully, free, of, the, legalese, that, often, clouds, these, documents, .)","[17.583139419555664, -9.901934623718262]","[7.111425876617432, -1.9578969478607178]","[-16.87145233154297, -4.008695125579834]"
"(of, course, ,, if, you, still, have, questions, about, anything, in, our, privacy, policy, ,, just, contact, us.you, should, read, our, entire, privacy, policy, ,, but, when, you, only, have, a, few, minutes, or, want, to, remember, something, later, on, ,, you, can, always, take, a, look, at, this, summary, —, so, you, can, review, some, of, the, basics, in, just, a, few, minutes.information)","[14.546968460083008, 0.3855372965335846]","[2.1271021366119385, -6.582714080810547]","[4.278918743133545, -18.665237426757812]"
"(of, course, ,, if, you, still, have, questions, about, anything, in, our, privacy, policy, ,, just, contact, us.you, should, read, our, entire, privacy, policy, ,, but, when, you, only, have, a, few, minutes, or, want, to, remember, something, later, on, ,, you, can, always, take, a, look, at, this, summary, —, so, you, can, review, some, of, the, basics, in, just, a, few, minutes.information)","[13.554123878479004, 0.15899385511875153]","[2.1344008445739746, -6.58812952041626]","[6.084807395935059, -17.989927291870117]"
"(of, course, ,, if, you, still, have, questions, about, anything, in, our, privacy, policy, ,, just, contact, us.you, should, read, our, entire, privacy, policy, ,, but, when, you, only, have, a, few, minutes, or, want, to, remember, something, later, on, ,, you, can, always, take, a, look, at, this, summary, —, so, you, can, review, some, of, the, basics, in, just, a, few, minutes.information)","[13.379727363586426, -0.8414849042892456]","[2.130619764328003, -6.584867000579834]","[6.700246810913086, -19.266883850097656]"


In [30]:
df['tsnetfidf'] = hero.tsne(df['TFIDF'])
hero.scatterplot(df, col='tsnetfidf', title="TF-IDF", hover_data= ['clean-text'])

In [32]:

hero.scatterplot(df, col='tsnec2v'
                 , title="C2V", hover_data = ['clean-text'])

In [34]:
hero.scatterplot(df, col='tsneuse'
                 , title="USE", hover_data = ['clean-text'])

In [36]:
instructions = df['Phrase'].tolist()

#check = 0
Bigger_list=[]

for sent in instructions:
    #check = check + 1
    #if check <2:
    li = list(str(sent).split())
    Bigger_list.append(li)

In [37]:
with open('train_data.pkl', 'wb') as f:
    pickle.dump(Bigger_list, f)

In [38]:
import pickle

In [39]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

In [40]:
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [41]:
model = Word2Vec()

In [42]:
model.build_vocab(train_data)

In [43]:
%%time
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 25.2 ms, sys: 577 µs, total: 25.8 ms
Wall time: 22.9 ms


(10523, 26815)

In [44]:
model.save('w2v.model')

In [45]:
%pwd

'/home/clifton/Desktop/Clifton_ISB/Ambiguty_Scoring'

In [48]:
search_term = 'can'
sim_terms = model.wv.most_similar(search_term, topn=5)
sim_terms

[('we', 0.9988677501678467),
 ('and', 0.998857855796814),
 ('to', 0.9988352060317993),
 ('your', 0.998799741268158),
 ('information', 0.9987935423851013)]

In [34]:
str(df['Phrase'][0])

'you use these services, you’ll share some information with us.'